In [1]:
import traceback
def prvar(__x):
    print(traceback.extract_stack(limit=2)[0][3][6:][:-1],"=",__x)

import numpy as np
import pandas as pd
import math
import csv
import os
import json

from scipy import sparse

In [8]:
## For TEST

# folder_name= 'data/kdd'
# course_name= 'sides_20_21'
# train_file= 'SIDES_20_21_6thyear_withspec_training_prepared.csv'
# test_file= 'SIDES_20_21_6thyear_withspec_test_prepared.csv'
# kc_col_name= 'specialty'
# min_interactions_per_user= 100
# remove_nan_skills= True
# verbose= True
# drop_duplicates= True
# remove_nan_answer_type= True
# min_answer_per_question= 5


In [1]:
def prepare_sides(folder_name, course_name, train_file, test_file, kc_col_name, min_interactions_per_user, min_answer_per_question, remove_nan_skills, remove_nan_answer_type, verbose, drop_duplicates=True):
    '''
    Reading input files
    Drop rows for which topic is not determined
    Return the pre-processed file and Q-matrix (for question-specialty matches).
    
    Arguments:
    folder_name -- path to the folder containig kdd files (algebra05, bridge_algebra06)
    course_name -- name of the course for which pre_processing is executed
    train_file -- original train_file provided by KDD cup organizers
    test_file -- original test_file provided by KDD cup organizers
    kc_col_name -- Skills id column
    min_interactions_per_user -- minimum number of interactions per student
    drop_duplicates -- if True, drop duplicates from dataset
    
    Outputs:
    data -- preprocessed dataset (pandas DataFrame)
    Q_mat -- corresponding q-matrix (item-skill relationships sparse array)
    '''
    
    if not os.path.exists(folder_name):
        print("The provided path for the data is invalid and the function will not be executed.")
        return None, None, None, None
        
    # reading csv file containing information about students' practice (attempt) history
    # from the train and test file provided by KDD organizer and then concatante them.
    train_file_path = folder_name  +'/'+  course_name  +'/'+ train_file
    df_train = pd.read_csv(train_file_path).rename(columns={
        'student': 'student',
        'question': 'question',
        kc_col_name: 'kc_id',
        'date_time': 'timestamp',
        'result': 'correct',
        'n_answer_option': 'n_options',
        'answer_option': 'answer_type',
        'type': 'quest_type',
    })[['student', 'question','correct', 'timestamp', 'kc_id','n_options','answer_type','quest_type']]
    
    if verbose:
        initial_shape = df_train.shape[0]
        print("Opened SIDES train data. Output: {} samples.".format(initial_shape))
        
    test_file_path = folder_name  +'/'+  course_name  +'/'+ test_file
    df_test = pd.read_csv(test_file_path).rename(columns={
        'student': 'student',
        'question': 'question',
        kc_col_name: 'kc_id',
        'date_time': 'timestamp',
        'result': 'correct',
        'n_answer_option': 'n_options',
        'answer_option': 'answer_type',
        'type': 'quest_type',
    })[['student', 'question', 'correct', 'timestamp', 'kc_id','n_options','answer_type','quest_type']]
    if verbose:
        initial_shape = df_test.shape[0]
        print("Opened SIDES test data. Output: {} samples.".format(initial_shape))
        
    #merge df_tarin & df_test    
    df_train['group'] = 0
    df_test['group'] = 1
    frames = [df_train, df_test]
    data = pd.concat(frames)
    del df_train
    del df_test
    
    # Remove potential duplicates
    #data= df_train.copy()
    initial_shape = data.shape[0]
    data = data[~data.duplicated()]
    if verbose:
        print("Removed {} duplicated samples.".format(initial_shape-data.shape[0]))
    initial_shape = data.shape[0]
    
    #removing rows with empty value for KC from our dataframe
    if remove_nan_skills:
        initial_shape=data.shape[0]
        data = data[~data["kc_id"].isnull()]
        if verbose:
            print("Removed {} samples with NaN skills.".format(initial_shape-data.shape[0]))
        initial_shape = data.shape[0]
    else:
        data.loc[data["kc_id"].isnull(), "kc_id"] = 'NaN'
        
        
    #removing rows with NAN value for n_correct_options from our dataframe
    if remove_nan_answer_type:
        initial_shape=data.shape[0]
        data = data.dropna(subset=["answer_type"])
        if verbose:
            print("Removed {} samples with NA answer_type.".format(initial_shape-data.shape[0]))
        initial_shape = data.shape[0]
            
    # Add correct time/date columns
    original_format = "%Y-%m-%dT%H:%M:%SZ"
    desired_format = "%Y%m%d%H%M%S"
    data["timestamp"] = pd.to_datetime(data["timestamp"])
    data["timestamp"] = data["timestamp"].apply(lambda x: x.strftime(desired_format))

    ## instead of the one above, we use the following code to get the correct time (if the time is in the format of integer)
    #data["correct_time"] = data["time"].apply(lambda x:str(x).zfill(6))
    #data["correct_date"] = data["date"].apply(lambda x:str(x))
    #full_dates = [str1+str2 for str1, str2 in zip(data["correct_date"],data["correct_time"])]
    #data["full_time"] = full_dates
    #data = data[["user_id","item_id","full_time","correct","group"]]

    data["timestamp"] = pd.to_datetime(data.loc[:,"timestamp"],format="%Y%m%d%H%M%S")
    data.loc[:,"timestamp"] = data.loc[:,"timestamp"] - data.loc[:,"timestamp"].min()
    data.loc[:,"timestamp"] = data.loc[:,"timestamp"].apply(lambda x: x.total_seconds()).astype(np.int64)


    # binarize correct column
    data.correct = data.correct.apply(lambda x:0 if x not in [0,1] else x).astype(np.int32)
    data = data[data['correct'].isin([0,1])] # Remove potential continuous outcomes
    if verbose:
        print("Removed {} samples with non-binary outcomes.".format(initial_shape-data.shape[0]))
    initial_shape = data.shape[0]
    data['correct'] = data['correct'].astype(np.int32) # Cast outcome as int32
    
    
    #removing users without enough interaction (min_interactions_per_user)
    data = data.groupby("student").filter(lambda x: len(x) >= min_interactions_per_user)
    if verbose:
        print('Removed {} samples '.format((initial_shape-data.shape[0])))
        print('(users with less than {} interactions).'.format((min_interactions_per_user)))
    initial_shape = data.shape[0]
        
    #removing questions without enough answers (min_answer_per_question)
    data = data.groupby("question").filter(lambda x: len(x) >= min_answer_per_question)
    if verbose:
        print('Removed {} samples '.format((initial_shape-data.shape[0])))
        print('(questions with less than {} answers).'.format((min_answer_per_question)))
    initial_shape = data.shape[0]
        
    # Create variables
    #data["item_id"] = data["pb_id"]
    data = data[['student', 'question', 'n_options','answer_type','kc_id', 'correct', 'timestamp', 'group']]
        
    # Transform ids into numeric
    data["user_id"] = np.unique(data["student"], return_inverse=True)[1].astype(np.int64)
    data["item_id"] = np.unique(data["question"], return_inverse=True)[1].astype(np.int64)
    
    # Rename questions/skills in item_skills data
    old_new_item_ids = data[~data.duplicated(["question","item_id"])][["question","item_id"]]
    old_new_user_ids = data[~data.duplicated(["student","user_id"])][["student","user_id"]]
    if not os.path.isdir(folder_name+'/'+ course_name+"/processed"):
        os.makedirs(folder_name+'/'+ course_name+"/processed")
    old_new_item_ids.to_csv(folder_name+'/'+ course_name+"/processed/old_new_item_ids.csv",index=False)
    old_new_user_ids.to_csv(folder_name+'/'+ course_name+"/processed/old_new_user_ids.csv",index=False)
    
    # # call skills data
    # spec_file= folder_name  +'/'+  course_name  +'/'+ 'questions_specialty.csv'
    # item_skills = pd.read_csv(spec_file)
    # item_skills = item_skills.merge(old_new_item_ids,on="question",how="right")
    
    # keep only the necessary columns
    data = data[['user_id', 'item_id', 'n_options','answer_type' ,'timestamp', 'correct', 'kc_id', 'group']]
    
    # To be safe, drop duplicates again
    data.drop_duplicates(inplace=True)
    if verbose:
        if data.shape[0] < initial_shape:
            logging.warning("{} duplicates have been found before saving the CSV.".format(initial_shape-data.shape[0]))
    initial_shape = data.shape[0]    
    
    # Create list of KCs
    listOfKC = []
    for kc_raw in data["kc_id"].unique():
        for elt in kc_raw.split('+'):
            listOfKC.append(elt)
    listOfKC = np.unique(listOfKC)

    dict1_kc = {}
    dict2_kc = {}
    for k, v in enumerate(listOfKC):
        dict1_kc[v] = k
        dict2_kc[k] = v
        
        
    # Build Q-matrix
    Q_mat = np.zeros((len(data["item_id"].unique()), len(listOfKC)))
    item_skill = np.array(data[["item_id","kc_id"]])
    for i in range(len(item_skill)):
        splitted_kc = item_skill[i,1].split('+')
        for kc in splitted_kc:
            Q_mat[item_skill[i,0],dict1_kc[kc]] = 1
    
    if verbose:
        print("Computed q-matrix. Shape: {}.".format(Q_mat.shape))
        
    print("Data preprocessing done. Final output: {} samples.".format((data.shape[0])))   


    # Save preprocessed data
    #data['timestamp'] =  pd.to_datetime(data['timestamp'])#, dayfirst=True)
    data.sort_values(by=["timestamp", "item_id"], inplace=True)#first, timestamp should be converted to datetime
    data.reset_index(inplace=True, drop=True) 
    sparse.save_npz(folder_name+'/'+ course_name+"/processed/q_mat.npz", sparse.csr_matrix(Q_mat))
    data.to_csv(folder_name+'/'+ course_name+"/processed/preprocessed_data.csv", index=False)
    
    # split train and test and save them
    train_set = data[data['group'] == 0]
    train_set.reset_index(inplace=True, drop=True)
    #train_set['timestamp'] =  pd.to_datetime(train_set['timestamp'])#, dayfirst=True)
    train_set.sort_values(by=["timestamp", "item_id"], inplace=True) #first, timestamp should be converted to datetime
    train_set.reset_index(inplace=True, drop=True)
    
    test_set = data[data['group'] == 1]
    test_set.reset_index(inplace=True, drop=True)
    #test_set['timestamp'] =  pd.to_datetime(test_set['timestamp'])#, dayfirst=True)
    test_set.sort_values(by=["timestamp", "item_id"], inplace=True)#first, timestamp should be converted to datetime
    test_set.reset_index(inplace=True, drop=True)
    
    train_set.to_csv(folder_name+'/'+ course_name+"/processed/train_set.csv", encoding='utf-8', index = False)
    test_set.to_csv(folder_name+'/'+ course_name+"/processed/test_set.csv", encoding='utf-8', index = False)
    
    # save skill_names_ids_map
    # Convert the NumPy array to a pandas DataFrame and first column as specilaty column and  indices as  specilat_id column
    skill_names_ids_map_df = pd.DataFrame(listOfKC,columns=['specialty'])
    # have a specialty id column with values from 0 to n_skills
    skill_names_ids_map_df['specialty_id'] = skill_names_ids_map_df.index
    # Save the DataFrame as a CSV file
    skill_names_ids_map_df.to_csv(folder_name+'/'+ course_name + '/processed/skill_names_ids_map.csv', index=False)

    listOfKC = list(listOfKC)
    # save listOfKC list
    with open(folder_name+'/'+ course_name+'/processed/listOfKC.json', 'w') as fp:
        json.dump(listOfKC, fp)
    
    # Save dict1_kc
    with open(folder_name+'/'+ course_name+'/processed/dict_of_kc.json', 'w') as fp:
        json.dump(dict1_kc, fp)
        
    # Save preprocessed data basic info
    with open( folder_name+'/'+ course_name+"/processed/config.json", 'w') as f:
        f.write(json.dumps({
            'n_users': data.user_id.nunique(),
            'n_items': data.item_id.nunique(),
            'n_skills': Q_mat.shape[1]
            }, indent=4))

    
    return data, Q_mat, listOfKC, dict1_kc, train_set, test_set, skill_names_ids_map_df

In [ ]:
# pre_processed_data, q_mat, listOfKC, dict_of_kc, train_set, test_set = prepare_sides('data/kdd', 'sides_20_21', \
 #                                                                  'SIDES_20_21_6thyear_withspec_training_prepared.csv', \
  #                                                                  'SIDES_20_21_6thyear_withspec_test_prepared.csv',\
   #                                                                 'specialty', 5, True, True, True)
 #
 
 #

